In [9]:
import rospy
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from math import radians, degrees
from actionlib_msgs.msg import *
from geometry_msgs.msg import Point

rospy.init_node('map_navi',anonymous = False)
ac = actionlib.SimpleActionClient('move_base', MoveBaseAction)

goal = MoveBaseGoal()
goal.target_pose.header.frame_id = 'map'
goal.target_pose.header.stamp = rospy.Time.now()

In [10]:
from geometry_msgs.msg import PoseWithCovarianceStamped, PoseStamped

current_pose = PoseWithCovarianceStamped()
processing = False

def callback(msg):
    global processing, current_pose
    if not processing:
        processing = True
        current_pose = msg.pose.pose

In [11]:
processing = False
rospy.Subscriber('/amcl_pose', PoseWithCovarianceStamped, callback)

In [12]:
current_pose

position: 
  x: -0.027586881213
  y: 3.11401899177
  z: 0.0
orientation: 
  x: 0.0
  y: 0.0
  z: -0.795833476712
  w: 0.60551554674

In [6]:
import tf
import math
import numpy as np
from geometry_msgs.msg import Quaternion

# function that converts from Quaternion to Euler
convert_to_nparray = lambda o: np.array([o.x, o.y, o.z, o.w])
def convert_to_euler(pose):
    tmp = convert_to_nparray(pose.orientation)
    tmp = tf.transformations.euler_from_quaternion(tmp)
    return tmp

# function that converts from Euler to Quaternion
convert_to_rad = lambda o: np.array(o)*math.pi/180.
def convert_to_quaternion(euler):
    p, q, r = convert_to_rad(euler)
    tmp = tf.transformations.quaternion_from_euler(p, q, r)
    return Quaternion(*tmp)

In [7]:
goal_pose = list(convert_to_euler(current_pose))
print(goal_pose)
print('===============================')
print(convert_to_quaternion(goal_pose))

[0.0, -0.0, 0.0007485988430896371]
x: 0.0
y: 0.0
z: 6.53275729422e-06
w: 0.999999999979


p, q, r = convert_to_euler(current_pose)
target_r = 45    # this is user input (orientation)

goal.target_pose.pose.orientation = convert_to_quaternion([0, 0, target_r])
goal.target_pose.pose.position = current_pose.position
goal.target_pose.header.stamp = rospy.Time.now()
goal

In [8]:
goal.target_pose.pose.position.x = current_pose.position.x 
goal.target_pose.pose.position.y = current_pose.position.y - 0.20

quat = convert_to_quaternion([0,0,0])
goal.target_pose.pose.orientation = quat
goal.target_pose.header.stamp = rospy.Time.now()
ac.send_goal(goal)